In [1]:
import os
import pandas as pd
import re
from os import walk
import numpy as np
import math

In [2]:
i = 0
filepaths = []

for root, dirs, files in os.walk("C:/Users/janja/OneDrive/Pulpit/DaneMGR", topdown=True):
    for name in dirs:
        if (bool(re.findall('\d$', name)) == False):
            Path = (root + '/' + name)
            filepaths.append(re.sub('DaneMGR\\\\', 'DaneMGR/', Path))
            

filenameList = []
fullPath = []
for Path in filepaths:
    for (dirpath, dirnames, filenames) in walk(Path):
        for name in filenames:
            if (bool(re.findall('fast_Unknown', name)) == True) and name not in filenameList:
                NewName = re.sub('._CsvLog', 'CsvLog', name)
                filenameList.append(NewName)
                fullPath.append(Path + '/' + NewName)
                   

In [16]:
print(fullPath)

['C:/Users/janja/OneDrive/Pulpit/DaneMGR/1/B/CsvLogBase_2022-01-15_191202.371_fast_Unknown.csv', 'C:/Users/janja/OneDrive/Pulpit/DaneMGR/1/R/CsvLogBase_2022-01-15_235231.870_fast_Unknown.csv', 'C:/Users/janja/OneDrive/Pulpit/DaneMGR/10/B/CsvLogBase_2022-03-22_190208.416_fast_Unknown.csv', 'C:/Users/janja/OneDrive/Pulpit/DaneMGR/10/R/CsvLogBase_2022-03-22_145436.972_fast_Unknown.csv', 'C:/Users/janja/OneDrive/Pulpit/DaneMGR/11/B/CsvLogBase_2022-03-24_155741.211_fast_Unknown.csv', 'C:/Users/janja/OneDrive/Pulpit/DaneMGR/11/R/CsvLogBase_2022-03-23_192319.765_fast_Unknown.csv', 'C:/Users/janja/OneDrive/Pulpit/DaneMGR/12/B/CsvLogBase_2022-04-03_181938.287_fast_Unknown.csv', 'C:/Users/janja/OneDrive/Pulpit/DaneMGR/12/R/CsvLogBase_2022-04-03_215403.654_fast_Unknown.csv', 'C:/Users/janja/OneDrive/Pulpit/DaneMGR/13/B/CsvLogBase_2022-04-04_185513.520_fast_Unknown.csv', 'C:/Users/janja/OneDrive/Pulpit/DaneMGR/13/R/CsvLogBase_2022-04-04_230427.597_fast_Unknown.csv', 'C:/Users/janja/OneDrive/Pulpit

In [3]:
def segmentation(pd_data, window_size):
    np_data = pd_data.to_numpy()
    nb_timestamps, nb_sensors = np_data.shape
    
    #window_size = 100 # Size of the data segments, earlier there was the value of 100
    timestamp_idx = 0 # Index along the timestamp dimension
    segment_idx = 0 # Index for the segment dimension
    
    nb_segments = int(math.floor(nb_timestamps/window_size))
    print(f'Starting segmentation with a window size of {window_size} resulting in {nb_segments} segments.')
    data_to_save = np.zeros((nb_segments,window_size,nb_sensors),dtype=np.float32)

    while segment_idx < nb_segments:
        data_to_save[segment_idx] = np_data[timestamp_idx:timestamp_idx+window_size,:]
        timestamp_idx += window_size
        segment_idx += 1
    return data_to_save

In [15]:
T = 'N' #500000 #'N' # describes which rows multiplied by n should be taken into the dataset #If chosen parameter is N the rows will not get dropped
fields = ['Infinity|RESP.ONLY_ONE_IN_GROUP [OHM]', 'Infinity|SPO2.SPO2_PULSE [COUNTS]']
WindowSize = 100
NumOfSensors = 2
df_0 = np.zeros((1,WindowSize, NumOfSensors))
df_1 = np.zeros((1,WindowSize, NumOfSensors))

files_total = len(fullPath)
i = 1
for path in fullPath:
    print(path)
    df_local = pd.read_csv(path, sep = ',', encoding = 'UTF-8', usecols=fields)
    df_local = df_local.interpolate()
    if T != 'N':
        df_local = df_local[df_local.index % T == 0] #Set to 2000 as 1 second is 20 observations
    match = re.findall("/B/Csv",path)
    
    df_segmented = segmentation(df_local, WindowSize)
    
    print(df_segmented)
    print(df_segmented.shape)
    
    neo = re.findall('\/([\d]{1,2})\/', path)
    neo2 = re.sub('\/', '', neo)
    if bool(match) == True:
        df_1 = np.concatenate(df_segmented, axis = 1)
        print(df_1.shape)
    else:
        df_0 = np.concatenate(df_segmented, axis = 0)
        print(df_0.shape)
    #neo = re.findall('\/([\d]{1,2})\/', path)
    #df_local.insert(0,'neonate', str(neo))
    #print(df_local)
    print(f"Imported file number: {i}, from files total: {files_total}, and that is {i*100/files_total:.2f}%")
    i+=1
 
    

C:/Users/janja/OneDrive/Pulpit/DaneMGR/1/B/CsvLogBase_2022-01-15_191202.371_fast_Unknown.csv
Starting segmentation with a window size of 100 resulting in 29864 segments.
[[[-1.5500e-01 -4.5000e+01]
  [-1.3350e-01 -5.2500e+01]
  [-1.1200e-01 -6.0000e+01]
  ...
  [-2.0900e-01 -7.3000e+01]
  [-2.1200e-01 -7.3000e+01]
  [-2.1500e-01 -7.3000e+01]]

 [[-2.1800e-01 -7.3000e+01]
  [-2.2025e-01 -7.1500e+01]
  [-2.2250e-01 -7.0000e+01]
  ...
  [-1.9750e-01  1.1500e+02]
  [-1.6900e-01  1.2400e+02]
  [-1.4050e-01  1.3100e+02]]

 [[-1.1200e-01  1.3800e+02]
  [-8.5500e-02  1.4250e+02]
  [-5.9000e-02  1.4700e+02]
  ...
  [-2.6175e-01  1.7500e+01]
  [-2.8150e-01  9.0000e+00]
  [-3.0125e-01  2.0000e+00]]

 ...

 [[-2.1660e+00  0.0000e+00]
  [-2.1660e+00  0.0000e+00]
  [-2.1660e+00  0.0000e+00]
  ...
  [-2.1660e+00  0.0000e+00]
  [-2.1660e+00  0.0000e+00]
  [-2.1660e+00  0.0000e+00]]

 [[-2.1660e+00  0.0000e+00]
  [-2.1660e+00  0.0000e+00]
  [-2.1660e+00  0.0000e+00]
  ...
  [-2.1660e+00  0.0000e+00]
  

KeyboardInterrupt: 

In [11]:
df_1.shape

(1935500, 2)

In [10]:
df_2 = np.zeros((1,WindowSize, NumOfSensors))
print(df_2.shape)

(1, 100, 2)
